In [ ]:
# Install necessary libraries
!pip install yfinance pandas_datareader

# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as smM
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
# Define the time period for analysis
start_date = "2022-01-01"
end_date = "2025-1-1"

# Specify the stock and benchmark tickers
stock_ticker = 'ASB'        # Stock of your choice
benchmark_ticker = '^GSPC'   # S&P 500 index symbol

# FRED symbol for the risk-free rate
risk_free_symbol = 'DGS3MO'  # 3-Month Treasury Bill
# Download stock data
stock_data = yf.download(stock_ticker, start=start_date, end=end_date, auto_adjust=True)

# Download benchmark data
benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date, auto_adjust=True)

# Display the first few rows of stock data
print("Stock Data Sample:")
print(stock_data.head())

# Display the first few rows of benchmark data
print("\nBenchmark Data Sample:")
print(benchmark_data.head())
# Calculate daily returns for the stock
stock_data['Daily_Return'] = stock_data['Close'].pct_change()

# Calculate daily returns for the benchmark
benchmark_data['Daily_Return'] = benchmark_data['Close'].pct_change()

# Drop the NaN values resulting from the percentage change calculation
stock_returns = stock_data['Daily_Return'].dropna()
benchmark_returns = benchmark_data['Daily_Return'].dropna()

In [ ]:
# Combine returns into one DataFrame
data = pd.concat([stock_returns, benchmark_returns], axis=1)
data.columns = ['Stock_Return', 'Benchmark_Return']

# Reset the index to ensure 'Date' is a column
data.reset_index(inplace=True)

# Display the combined data
print("\nCombined Returns Data:")
print(data.head())

In [ ]:
# Fetch the risk-free rate data
risk_free_data = web.DataReader(risk_free_symbol, 'fred', start_date, end_date)

# Forward-fill any missing data (e.g., weekends and holidays)
risk_free_data = risk_free_data.ffill()

# Convert the annualized rates from percentage to decimal
risk_free_data['Risk_Free_Rate'] = risk_free_data[risk_free_symbol] / 100

# Calculate the daily risk-free rate
trading_days = 252  # Approximate number of trading days in a year
risk_free_data['Daily_Risk_Free_Rate'] = risk_free_data['Risk_Free_Rate'] / trading_days

# Reset the index to have 'DATE' as a column
risk_free_data.reset_index(inplace=True)

# Display the risk-free rate data
print("\nRisk-Free Rate Data Sample:")
print(risk_free_data.head())

In [ ]:
# Merge the risk-free data with the returns data
data = pd.merge(data, risk_free_data[['DATE', 'Daily_Risk_Free_Rate']], left_on='Date', right_on='DATE', how='left')

# Drop the redundant 'DATE' column
data.drop(['DATE'], axis=1, inplace=True)

# Forward-fill any missing daily risk-free rates
data['Daily_Risk_Free_Rate'].fillna(method='ffill', inplace=True)

# Display the merged data
print("\nData After Merging Risk-Free Rates:")
print(data.head())

In [ ]:
# Calculate the stock's excess returns
data['Stock_Excess_Return'] = data['Stock_Return'] - data['Daily_Risk_Free_Rate']

# Calculate the benchmark's excess returns
data['Benchmark_Excess_Return'] = data['Benchmark_Return'] - data['Daily_Risk_Free_Rate']

# Display the excess returns
print("\nExcess Returns Data:")
print(data[['Date', 'Stock_Excess_Return', 'Benchmark_Excess_Return']].head())

In [ ]:
# Define the dependent and independent variables
X = data['Benchmark_Excess_Return']  # Independent variable
y = data['Stock_Excess_Return']      # Dependent variable

# Add a constant term to the independent variable
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Output the regression results
print("\nCAPM Regression Results:")
print(model.summary())

In [ ]:
# Extract the regression coefficients
alpha = model.params['const']
beta = model.params['Benchmark_Excess_Return']
r_squared = model.rsquared

# Display the regression parameters
print(f"\nRegression Parameters:")
print(f"Alpha (Intercept): {alpha:.6f}")
print(f"Beta (Slope): {beta:.6f}")
print(f"R-squared: {r_squared:.4f}")

In [ ]:
# Interpret the beta value
if beta > 1:
    volatility = "more volatile than the market"
elif beta < 1:
    volatility = "less volatile than the market"
else:
    volatility = "equally volatile as the market"

print(f"\nBeta Interpretation:")
print(f"The stock {stock_ticker} has a beta of {beta:.4f}, which means it is {volatility}.")

In [ ]:
# Set the plot size
plt.figure(figsize=(12, 6))

# Create a scatter plot with a regression line
sns.regplot(x='Benchmark_Excess_Return', y='Stock_Excess_Return', data=data,
            line_kws={'color': 'red'})

# Customize the plot
plt.title(f'CAPM Regression: {stock_ticker} Excess Returns vs. {benchmark_ticker}')
plt.xlabel('Benchmark Excess Returns')
plt.ylabel('Stock Excess Returns')

# Show the plot
plt.show()

In [ ]:
# Print a summary of findings
print("\nSummary of Findings:")
print(f"- The stock {stock_ticker} has an alpha of {alpha:.6f}, suggesting {'outperformance' if alpha > 0 else 'underperformance'} relative to the CAPM prediction.")
print(f"- A beta of {beta:.4f} indicates that the stock is {volatility}.")
print(f"- An R-squared of {r_squared:.4f} means that {r_squared*100:.2f}% of the stock's excess returns are explained by the market's excess returns.")

Copilot was used for assistance with code generation